In [1]:
# Import necessary libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Activation, Input, Add
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

In [2]:
# Set the image dimensions and channels
h, w = 50, 50  # Height and Width of images
n_channels = 1  # Since images are grayscale, 1 channel
n_classes = 4  # Number of output classes for classification
l2_lambda = 0.0001  # Regularization parameter for L2 regularization

# Load your preprocessed image data
X = np.load('X.npy')  # The X.npy contains the flattened grayscale image data
y = np.load('y.npy', allow_pickle=True)  # y.npy contains the labels for the images
X1 = X.reshape(-1, h, w, n_channels)  # Reshaping to match the input required for CNNs (height, width, channels)

# Data Augmentation for improving model generalization
datagen = ImageDataGenerator(
    rotation_range=5,  # Random rotations
    width_shift_range=0.1,  # Horizontal shifts
    height_shift_range=0.1,  # Vertical shifts
    horizontal_flip=True,  # Flip images horizontally
    vertical_flip=True,  # Flip images vertically
    fill_mode='nearest'  # Strategy to fill in pixels after augmentation
)

datagen.fit(X1)  # Fit the generator to the data

# Label encoding (from string labels to numeric values) for multi-class classification
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)  # Transform string labels to integer
y_ohe = to_categorical(y_encoded, num_classes=n_classes)  # One-hot encode the integer labels for model training


In [3]:
# Define AlexNet1 Model
def create_alexnet1():
    model = Sequential(name='alexnet1')
    num_filt_layers = [32, 64, 128]  # Number of filters in each Conv layer
    for num_filters in num_filt_layers:
        model.add(Conv2D(filters=num_filters, kernel_size=(3, 3), padding='same', activation='relu', input_shape=(h, w, n_channels)))
        model.add(MaxPooling2D(pool_size=(2, 2)))  # Max pooling layer to reduce spatial dimensions
    model.add(Flatten())  # Flatten the output from conv layers to feed into fully connected layers
    model.add(Dense(128, activation='relu'))  # Fully connected layer with 128 neurons
    model.add(Dense(128, activation='relu'))  # Second fully connected layer
    model.add(Dense(n_classes, activation='softmax'))  # Output layer with softmax activation
    return model


In [4]:
# Define AlexNet2 Model
def create_alexnet2():
    model = Sequential(name='alexnet2')
    num_filt_layers = [64, 128, 256]  # Higher number of filters in this version
    for num_filters in num_filt_layers:
        model.add(Conv2D(filters=num_filters, kernel_size=(3, 3), padding='same', activation='relu', input_shape=(h, w, n_channels)))
        model.add(MaxPooling2D(pool_size=(3, 3)))  # Larger pooling window for more aggressive downsampling
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))  # Larger fully connected layer with 256 neurons
    model.add(Dense(256, activation='relu'))
    model.add(Dense(n_classes, activation='softmax'))  # Output layer for multi-class classification
    return model


In [5]:
# Define ResNet1 Model with residual connections
def create_resnet1():
    input_holder = Input(shape=(h, w, n_channels))  # Input layer
    x = Conv2D(32, (3, 3), padding='same', activation='relu')(input_holder)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(32, (3, 3), padding='same', activation='relu')(x)
    x_split = MaxPooling2D((2, 2))(x)  # This layer will be added back via residual connections
    x = Conv2D(64, (1, 1), padding='same', activation='linear')(x_split)
    x = Conv2D(64, (3, 3), padding='same', activation='linear')(x)
    x = Conv2D(32, (1, 1), padding='same', activation='relu')(x)
    x = Add()([x, x_split])  # Adding residual connection
    x = Activation('relu')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Flatten()(x)
    x = Dropout(0.25)(x)  # Dropout for regularization
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(n_classes, activation='softmax')(x)
    return Model(inputs=input_holder, outputs=x, name='resnet1')


In [6]:
# Define ResNet2 Model with deeper architecture
def create_resnet2():
    input_holder = Input(shape=(h, w, n_channels))
    x = Conv2D(64, (3, 3), padding='same', activation='relu')(input_holder)
    x = MaxPooling2D((3, 3))(x)
    x = Conv2D(64, (3, 3), padding='same', activation='relu')(x)
    x_split = MaxPooling2D((3, 3))(x)
    x = Conv2D(64, (1, 1), padding='same', activation='linear')(x_split)
    x = Conv2D(64, (3, 3), padding='same', activation='linear')(x)
    x = Conv2D(64, (1, 1), padding='same', activation='relu')(x)
    x = Add()([x, x_split])  # Adding the residual connection
    x = Activation('relu')(x)
    x = MaxPooling2D((3, 3))(x)
    x = Flatten()(x)
    x = Dropout(0.25)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(n_classes, activation='softmax')(x)
    return Model(inputs=input_holder, outputs=x, name='resnet2')


In [7]:
# Define MLP model
def create_mlp():
    model = Sequential(name='mlp')
    model.add(Flatten(input_shape=(h, w, n_channels)))  # Flatten the input (50x50x1) into a vector
    model.add(Dense(30, activation='relu'))  # Dense layer with 30 neurons
    model.add(Dense(15, activation='relu'))  # Dense layer with 15 neurons
    model.add(Dense(n_classes, activation='softmax'))  # Output layer with softmax activation
    return model



In [8]:
def train_and_evaluate(model, X, y, n_splits=10, batch_size=32, epochs=20):
    skf = StratifiedKFold(n_splits=n_splits)  # Stratified K-fold cross-validation
    precision_all = np.empty((0, n_classes), float)  # Placeholder for precision scores
    history_all = []  # To store history for each fold

    # Iterate over each fold
    for train_index, test_index in skf.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        # Augment training data
        datagen.fit(X_train)
        
        # Compile the model
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        
        # Train the model and save the history
        history = model.fit(datagen.flow(X_train, to_categorical(y_train, n_classes), batch_size=batch_size),
                            validation_data=(X_test, to_categorical(y_test, n_classes)),
                            steps_per_epoch=len(X_train) // batch_size,
                            epochs=epochs,
                            callbacks=[
                                EarlyStopping(monitor='val_loss', patience=5),  # Stop early if no improvement
                                ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3)  # Reduce learning rate on plateau
                            ])
        
        # Save the training history for this fold
        history_all.append(history)

        # Predict the test set
        y_pred = np.argmax(model.predict(X_test), axis=1)
        
        # Calculate precision scores
        precision_scores = precision_score(y_test, y_pred, average=None)
        precision_all = np.append(precision_all, [precision_scores], axis=0)
        print(f"Precision: {precision_scores * 100}")  # Print precision for each class
    
    # Calculate mean precision across all folds
    precision_mean = np.mean(precision_all, axis=0)
    print("Mean precision scores over 10 folds: ", precision_mean)
    
    return precision_all, precision_mean, history_all  # Return history_all too


In [9]:
# Instantiate models
alexnet1 = create_alexnet1()
alexnet2 = create_alexnet2()
resnet1 = create_resnet1()
resnet2 = create_resnet2()
mlp = create_mlp()

# Train and evaluate each model
models = [alexnet1, alexnet2, resnet1, resnet2, mlp]
model_names = ["AlexNet1", "AlexNet2", "ResNet1", "ResNet2", "MLP"]
precision_results = {}

# Train and evaluate each model, store the precision results and history
for model, name in zip(models, model_names):
    print(f"Training {name} model")
    precision_all, precision_mean, history_all = train_and_evaluate(model, X1, y_encoded)
    precision_results[name] = (precision_all, precision_mean, history_all)


Training AlexNet1 model
Epoch 1/20


/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(
/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


 5/19 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.2320 - loss: 1.3905

/opt/anaconda3/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - accuracy: 0.3401 - loss: 1.3264 - val_accuracy: 0.4429 - val_loss: 1.3553 - learning_rate: 0.0010
Epoch 2/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3438 - loss: 0.5655 - val_accuracy: 0.4429 - val_loss: 1.2947 - learning_rate: 0.0010
Epoch 3/20
 3/19 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.4670 - loss: 1.0544

...

2024-10-21 22:31:59.460202: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-10-21 22:31:59.558593: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6917 - loss: 0.7556 - val_accuracy: 0.7429 - val_loss: 0.7733 - learning_rate: 0.0010
Epoch 8/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 561us/step - accuracy: 0.6875 - loss: 0.5189 - val_accuracy: 0.6714 - val_loss: 0.7937 - learning_rate: 0.0010
Epoch 9/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6938 - loss: 0.7732 - val_accuracy: 0.7000 - val_loss: 0.7451 - learning_rate: 0.0010
Epoch 10/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 489us/step - accuracy: 0.6250 - loss: 0.3852 - val_accuracy: 0.7143 - val_loss: 0.7464 - learning_rate: 0.0010
Epoch 11/20
 1/19 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.6875 - loss: 0.6748

2024-10-21 22:31:59.665452: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-10-21 22:31:59.769519: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6900 - loss: 0.7904 - val_accuracy: 0.6429 - val_loss: 0.8696 - learning_rate: 0.0010
Epoch 12/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 494us/step - accuracy: 0.5938 - loss: 0.4219 - val_accuracy: 0.6429 - val_loss: 0.8707 - learning_rate: 0.0010
Epoch 13/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7058 - loss: 0.7283 - val_accuracy: 0.7000 - val_loss: 0.7875 - learning_rate: 1.0000e-04
Epoch 14/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 483us/step - accuracy: 0.7188 - loss: 0.4959 - val_accuracy: 0.7143 - val_loss: 0.7847 - learning_rate: 1.0000e-04
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
Precision: [ 66.66666667  58.33333333 100.          65.38461538]
Mean precision scores over 10 folds:  [0.7199062  0.54748834 0.88675439 0.69400504]


2024-10-21 22:31:59.869469: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-10-21 22:31:59.968495: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


In [ ]:
# Precision curves for all models
plt.figure(figsize=(12, 8))

# Iterate over the models in precision_results
for model_name in precision_results:
    precision_data = precision_results[model_name]
    
    # Unpack the precision data
    precision_all = precision_data[0]  # Assuming the first element is precision_all
    precision_mean = precision_data[1]  # Assuming the second element is precision_mean

    # Plot the mean precision for each model
    plt.plot(np.mean(precision_all, axis=0), label=f"{model_name} Mean Precision")

# Customize the plot
plt.title("Precision Curves for All Models", fontsize=16)
plt.xlabel("Classes", fontsize=12)
plt.ylabel("Precision", fontsize=12)
plt.legend(loc="best", fontsize=12)
plt.grid(True)
plt.show()


In [ ]:
# Accuracy and loss curves for each model
for model_name in precision_results:
    # Assuming the third element in precision_results is the history data
    precision_all, precision_mean, history_all = precision_results[model_name]

    # Plot accuracy and loss for each fold
    for i, history in enumerate(history_all):
        epochs_range = range(len(history.history['loss']))  # Adjust epochs range based on the actual history length

        plt.figure(figsize=(12, 5))

        # Plot accuracy
        plt.subplot(1, 2, 1)
        plt.plot(epochs_range, history.history['accuracy'], label=f"Train accuracy fold {i+1}")
        plt.plot(epochs_range, history.history['val_accuracy'], label=f"Validation accuracy fold {i+1}")
        plt.legend()
        plt.title(f"{model_name} Accuracy fold {i+1}")

        # Plot loss
        plt.subplot(1, 2, 2)
        plt.plot(epochs_range, history.history['loss'], label=f"Train loss fold {i+1}")
        plt.plot(epochs_range, history.history['val_loss'], label=f"Validation loss fold {i+1}")
        plt.legend()
        plt.title(f"{model_name} Loss fold {i+1}")

        plt.show()
